# Basic syntax and workflow: Random walk

An important model in science (biology, chemistry, physics, etc.) is Brownian motion, or random walk, a model of a molecule moving in a medium.

The simplest model we can make is a particle jumping at random on the integers. We save the list of consecutive positions in an array.

We wrap the code in a function in order to make it more general (e.g. to be able to change the number of steps). It turns out that this also makes it (much) more efficient in Julia:

In [47]:
"""
    walk(T)

Simulate a simple random walk up to time `T`.
Returns the trajectory as a `Vector`.
"""
function walk(T::Integer)

    x = 0  # position
    trajectory = [x]  # initialise vector

    for t in 1:T  # range

        if rand() < 0.5
            x += 1       # equivalent to `x = x + 1`, i.e. modifies the value of x
        else
            x -= 1
        end
        
        push!(trajectory, x)  # add the value of `x` to the Vector `trajectory`

    end
    
    return trajectory

end

walk

In [50]:
supertype(Signed)

Integer

In [48]:
?walk

search: walkdir WalleniusNoncentralHypergeometric showall rowvals wsample



```
walk(T)
```

Simulate a simple random walk up to time `T`. Returns the trajectory as a `Vector`.


We call the function and give the calculated data a name:

In [51]:
traj = walk(10)

11-element Array{Int64,1}:
 0
 1
 2
 3
 2
 3
 2
 1
 2
 3
 2

We can use `show(traj)` to use less vertical space:

In [52]:
show(traj)

[0, 1, 2, 3, 2, 3, 2, 1, 2, 3, 2]

## Visualization

We wish to plot the trajectory $x(t)$ as a function of $t$. 

We recommen the `Plots.jl` package, which provides a common interface to several plotting "backends":

In [61]:
# Pkg.add("Plots")    # install the Plots.jl package if you don't already have it -- do once in your life
# Pkg.add("GR")

using Plots           # load the package in each session
gr()   # choose the GR backend

Plots.GRBackend()

In [56]:
T= 10
collect(1:2:T)

5-element Array{Int64,1}:
 1
 3
 5
 7
 9

In [57]:
T = 10
plot(1:T, walk(T), marker=:square)  # vector of x coordinates, and vector of y coordinates

hline!([0], c=:black, ls=:dash, lw=2)  # add dashed horizontal line to plot

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

In [58]:
# Pkg.add("Plotly")

plotlyjs()

Plots.PlotlyJSBackend()

In [59]:
T = 10
plot(1:T, walk(T), marker=:square)  # vector of x coordinates, and vector of y coordinates

hline!([0], c=:black, ls=:dash, lw=2)  # add dashed horizontal line to plot

Let's draw a few walks:

In [60]:
T = 30  # maximum time
N = 10  # number of walks

plot(legend=false)  # make empty plot with no legend

for i in 1:N
    plot!(1:T, walk(T))   # ! means "add to plot"
end

hline!([0], c=:black, ls=:dash, lw=2)  # add

## Data analysis

Now that we have a feel for the behaviour of the system, let's collect some data and do some data analysis.
Let's calculate the variance of the positions of `N` walks after time `T`:

A simple implementation could be as follows. First let's rewrite the `walk` function to only return the last value:

In [62]:
function walk(T)

    x = 0  

    for t in 1:T  

        if rand() < 0.5
            x += 1 
        else
            x -= 1
        end
        
    end
    
    return x

end

walk (generic function with 1 method)

In [63]:
function walks(N, T)
    
    sumsq = 0.0
    
    for i in 1:N
        w = walk(T)
        sumsq += w^2
    end 
    
    sumsq /= N
    
    return sumsq
end

walks (generic function with 2 methods)

In [65]:
@time walks(100000, 10000)
@time walks(100000, 10000)

  2.069897 seconds (5 allocations: 176 bytes)
  2.041898 seconds (5 allocations: 176 bytes)


9984.88528

Note the memory allocation in the first run. This is due to the function being compiled. 
The first run should always be discarded for benchmarking purposes.  

For microbenchmarks, prefer the `BenchmarkTools.jl` package:

In [66]:
# Pkg.add("BenchmarkTools")

using BenchmarkTools, Compat

In [67]:
@benchmark walks(10, 10)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     238.436 ns (0.00% GC)
  median time:      248.388 ns (0.00% GC)
  mean time:        265.825 ns (0.00% GC)
  maximum time:     836.772 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     417

## Vectorized (Matlab-ish) style:

We can easily try out different simulation methods for our `walk` function.
To do so, we write a generic `walks` function that **takes a function as argument**.

We also use **array comprehensions** or **generator expressions**:

In [70]:
rand()

0.8311029316173111

In [71]:
[rand() < 0.5 for i in 1:10]  # array comprehension

10-element Array{Bool,1}:
 false
  true
 false
  true
  true
 false
 false
  true
  true
  true

In [72]:
(rand() < 0.5 for i in 1:10)  # generator expression

Base.Generator{UnitRange{Int64},##19#20}(#19, 1:10)

In [73]:
function walk2(T) 
    right_steps = sum(rand() < 0.5 for i in 1:T)   # generator
    left_steps = T - right_steps
    
    return right_steps - left_steps
end
  
walks(f, N, T) = var(f(T) for i in 1:N)  # generator

walks (generic function with 2 methods)

In [74]:
N = 10^5
T = 10^4

@time walks(walk, N, T)

  2.144858 seconds (42.41 k allocations: 2.246 MiB)


10044.027305037496

In [76]:
@time walks(walk2, N, T)

  6.156844 seconds (300.01 k allocations: 7.630 MiB)


10007.447590365573

In [77]:
function walk3(T) 
    right_steps = sum(rand(Bool, T))   # generator
    left_steps = T - right_steps
    
    return right_steps - left_steps
end

walk3 (generic function with 1 method)

In [79]:
@time walks(walk3, 100000, 10000) 

  2.648700 seconds (100.01 k allocations: 970.459 MiB, 5.82% gc time)


9992.79033009489

In [80]:
function walk4(T) 
    right_steps = sum(bitrand(T))
    left_steps = T - right_steps
    
    return right_steps - left_steps
end

walk4 (generic function with 1 method)

In [81]:
@time walks(walk4, 100000, 10000)  

  0.197461 seconds (643.03 k allocations: 153.388 MiB, 12.65% gc time)


10001.509872583183

Of course, it's *possible* to do these things in C++, but it's certainly much *harder*.

## Look at the data

It is easy to collect data:

In [82]:
walk_data(N, T) = [walk(T) for i in 1:N]

walk_data (generic function with 1 method)

In [83]:
using Plots; gr()

Plots.GRBackend()

In [86]:
data = walk_data(10000, 10000)

histogram(data, bins=100, normed=true, ylim=(0,0.006), xlim=(-300,300))


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -300 
 
 
 -200 
 
 
 -100 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 0.000 
 
 
 0.001 
 
 
 0.002 
 
 
 0.003 
 
 
 0.004 
 
 
 0.005 
 
 
 0.006 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

Let's compare this to the known analytical result: a normal distribution with variance equal to the number of steps:

We can use Unicode variable names and operators:

In [87]:
μ = 0        # type \mu<TAB>
σ = √10000   # type \sqrt<TAB>

100.0

We will use the probability density function (PDF) from the `Distributions.jl` package:

In [41]:
# Pkg.add("Distributions")
using Distributions

In [88]:
histogram(data, bins=100, normed=true, label="data")
plot!(-250:250, x->pdf(Normal(0, σ), x), label="exact", lw=3)  # probability 
xaxis!("distance")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -400 
 
 
 -200 
 
 
 0 
 
 
 200 
 
 
 400 
 
 
 0.000 
 
 
 0.001 
 
 
 0.002 
 
 
 0.003 
 
 
 0.004 
 
 
 distance 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 exact

To save the figure, we use `savefig`:

In [89]:
savefig("test.pdf")

Now we can open the file. `;` sends the command to the command line (shell); on Mac, `open` opens the file:

In [90]:
;open test.pdf

## Summary:

- Julia allows us to write **readable** code 

- Julia allows us to calculate **fast**

- **And** interactively visualize our data

- All in one place

# Save the data

Suppose we wish to save our data to a file on disk.

The good solution is to use the `JLD.jl` ("JuLia Data") package (which uses the standard HDF5 format underneath). We will also save another variable for illustrative purposes.

In [ ]:
# Pkg.add("JLD")

using JLD

In [72]:
i = 17

17

In [73]:
save("random_walk.jld", "final_pos", data, "arbitrary", i)

In [74]:
all = load("random_walk.jld")

Dict{String,Any} with 2 entries:
  "arbitrary" => 17
  "final_pos" => [-20, 56, -24, 136, -26, 46, -100, -60, 14, 160  …  42, 102, -…

The data is read in as a **dictionary**, associating to each variable name its value. We can extract these as

In [75]:
j = all["arbitrary"]

17

Alternatively, we can *read only part of the data file*:

In [77]:
final_pos = load("random_walk.jld", "final_pos")

10000-element Array{Int64,1}:
  -20
   56
  -24
  136
  -26
   46
 -100
  -60
   14
  160
 -108
 -118
  -68
    ⋮
  -76
  110
   42
  102
  -20
   64
   38
    6
   -6
  -24
  124
    4

Note that the type of the data has now been correctly preserved. This is the point of using `JLD`. 

# Workflow: Extract code into a Julia script

Since we have developed a quantity of code in this notebook, it is a good moment to extract the code into a Julia script file, with termination `.jl`, that we edit, e.g. with Juno.

We can then read in the file as if we typed it with

In [ ]:
include("random_walk.jl")

Later on, we could make it into a `module` (separate namespace) and then into a Julia pacakge.